In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import pathlib
model_dir = pathlib.Path().resolve().parent/"models"
model_name = "Qwen/Qwen2.5-0.5B-Instruct"\

model_dir

WindowsPath('C:/Users/egor/files/projects/FPA_NLP/models')

In [2]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
    cache_dir=model_dir
)



In [3]:
model = model.to("cuda")
model.device

device(type='cuda', index=0)

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [5]:
prompt = """ Расскажи суть следующего диалога
А. – Алло. Центр «Интерконгресс» .

Б. – Здравствуйте. Радитехнический университет. Миронова Ольга. Я вам вчера звонила относительно симпозиума.

А. – Добрый день. Слушаю вас.

Б. – Вы можете предоставить нам помещение на сто тридцать – сто сорок человек?

А. – Да. Мы можем забронировать для вас конференц-зал на сто пятьдесят мест.

Б. – Это нам подходит, спасибо. А как оформить аренду мебели?

А. – В заявке нужно точно указать названия всех предметов и их количество.

Б. – Как отправить вам заявку?

А. – Вы должны отправить на наш адрес гарантийное письмо. В нем вы и укажете все виды услуг и их стоимость.

Б. – Понятно. Письмо можно отправить и по почте, и по факсу.

А. – Да.

Б. – И через сколько дней вы его получите?

А. – Письмо идет обычно два-три дня.

Б. – Это довольно долго.

А. – Можно послать с нарочным, тогда оно придет к нам в течение двух часов.

Б. – Так мы и сделаем. Спасибо большое за исчерпывающую информацию.

А. – Пожалуйста. Всего доброго."""
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]

In [6]:
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

In [ ]:
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)

<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
 Расскажи суть следующего диалога
А. – Алло. Центр «Интерконгресс» .

Б. – Здравствуйте. Радитехнический университет. Миронова Ольга. Я вам вчера звонила относительно симпозиума.

А. – Добрый день. Слушаю вас.

Б. – Вы можете предоставить нам помещение на сто тридцать – сто сорок человек?

А. – Да. Мы можем забронировать для вас конференц-зал на сто пятьдесят мест.

Б. – Это нам подходит, спасибо. А как оформить аренду мебели?

А. – В заявке нужно точно указать названия всех предметов и их количество.

Б. – Как отправить вам заявку?

А. – Вы должны отправить на наш адрес гарантийное письмо. В нем вы и укажете все виды услуг и их стоимость.

Б. – Понятно. Письмо можно отправить и по почте, и по факсу.

А. – Да.

Б. – И через сколько дней вы его получите?

А. – Письмо идет обычно два-три дня.

Б. – Это довольно долго.

А. – Можно послать с нарочным, тогда оно придет к нам в 

In [11]:
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

In [12]:
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

Суть диалога заключается в обсуждении вопросов по организации и оформлению аренду мебели для проведения конференции или мероприятия. Важно отметить, что:

1. Предоставление помещений - это ключевой момент для успешного проведения мероприятия.
2. Обеспечение достоверной информации о возможных условиях аренды (название предметов, количество).
3. Возможность отправки письма с гарантией.
4. Контактная информация (почта или факс).

Общие принципы:
- Уточнение деталей согласно заявке
- Отправка письма на указанные сроки
- Прибытие объекта в течение определенного времени

Таким образом, диалог показывает необходимость четкой и подробной подготовки для успешного проведения мероприятий, включая вопросы аренду мебели.
